This notebook can be used to calculate NA Corrected intensities as well as fractional enrichment for LCMS data with ppm varying with molecular mass. The example shows a dataset with D2 label:

 - D_Sample_Input_Simple.xlsx - demo raw MS intensity file containing intensities for C10H17N3O6S simulated using combinatorics by considering C13 indistinguishable with O17 and N15 indistinguishable with S34

In [1]:
import pandas as pd
import numpy as np
import re

from corna.inputs import maven_parser as parser
import corna.constants as const
from corna.helpers import replace_negatives_in_column, merge_multiple_dfs
from corna.algorithms.nacorr_lcms import na_correction
from corna.postprocess import fractional_enrichment


Reading raw file and merging with sample metadata if present, in this example running without sample metadata

In [2]:
raw_df = pd.read_excel('D_Sample_Input_Simple_one.xlsx')
#raw_df = pd.read_excel('C_Sample_Input_Simple_three.xlsx')
sample_metadata = pd.DataFrame()

merged_df, iso_tracer_data, element_list = parser.read_maven_file(raw_df, sample_metadata)
merged_df.head()

,Name,Label,Formula,Sample,Intensity,Unlabeled Fragment
0,glucose-6-phosphate,C12 PARENT,C6H10O5,1_1,12658.33,glucose-6-phosphate
1,glucose-6-phosphate,H2-label-1,C6H10O5,1_1,85214.52,glucose-6-phosphate


According to the formula from Su, Xiaoyang et al.,2017

\begin{equation*}
\frac{Δm}{m} = 1.66 × \frac{m^\frac{1}{2}}{(MinimalNominalResolution×√200)} × 10^6
\end{equation*}

Different vendors will have different formulas for the above, the function below converts resolution to ppm according to the formula above but it can be modified as per usage

Dictionary containing natural abundance values for the common isotopes found in nature. It can be defined by the user or one can use the default values from the package. The format of the dictionary is as shown below: 

{E:[M0, M1, ..Mn]} where E is the element symbol and the natural abundance fraction is in the increasing order of masses. For example:

In [3]:
#user defined
#from accucor
na_dict={'C':[0.9893, 0.0107],
           'H':[0.999885, 0.000115],
           'N':[0.99636, 0.00364],
           'O':[0.99757, 0.00038, 0.00205],
           'S':[0.9493, 0.00762, 0.0429]}

Performing na_correction and using the dictionary above for NA values. For Orbitrap, for molecular mass of 307 (which is our input compound) and 293808 resolution, ppm is ~7, according to the formula from Su, Xiaoyang et al.,2017

\begin{equation*}
\frac{Δm}{m} = 1.66 × \frac{m^\frac{1}{2}}{(MinimalNominalResolution×√200)} × 10^6
\end{equation*}

which is our ppm_user_input 

In [4]:
na_corr_df, ele_corr_dict = na_correction(merged_df, iso_tracers=['H2'], eleme_corr={},
                                          na_dict=na_dict, autodetect=True, res=100000, 
                                          res_mw=200, instrument='orbitrap')
na_corr_df = replace_negatives_in_column(na_corr_df, const.NA_CORRECTED_WITH_ZERO, const.NA_CORRECTED_COL)
na_corr_df

['C', 'O17', 'O18']
<function <lambda> at 0x1208a46e0>
0.0029219079
4132.20178018509
3427.249736581977
0.8294003823860855
0.0
0.0020596653
2912.80660120925
3427.249736581977
1.176614243856477
1.0
0.008307111
11748.029040338726
3427.249736581977
0.29172976375985876
0.0
[('C', 0.0), ('O17', 1.0), ('O18', 0.0)]
{'H': ['C', 'O17', 'O18']}
{'O17': 1.0, 'C': 0.0, 'O18': 0.0}
C
{'H': [0.999885, 0.000115], 'C': [0.9893, 0.0107], 'S': [0.9493, 0.00762, 0.0429], 'O': [0.99757, 0.00038, 0.00205], 'N': [0.99636, 0.00364]}
0.0
[[0.93749304 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.        ]
 [0.         0.93749304 0.         0.         0.         0.
  0.         0.         0.         0.         0.        ]
 [0.         0.         0.93749304 0.         0.         0.
  0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.93749304 0.         0.
  0.         0.         0.         0.         0.        ]
 [0.    

,Name,Formula,Indistinguishable_isotope,Sample,NA Corrected,Intensity,Label,NA Corrected with zero
0,glucose-6-phosphate,C6H10O5,"{'H': ['C', 'O17', 'O18']}",1_1,1.368330e+04,12658.33,C12 PARENT,1.368330e+04
1,glucose-6-phosphate,C6H10O5,"{'H': ['C', 'O17', 'O18']}",1_1,9.206214e+04,85214.52,H2-label-1,9.206214e+04
2,glucose-6-phosphate,C6H10O5,"{'H': ['C', 'O17', 'O18']}",1_1,-2.706464e+02,0.00,H2-label-2,0.000000e+00
3,glucose-6-phosphate,C6H10O5,"{'H': ['C', 'O17', 'O18']}",1_1,5.391071e-01,0.00,H2-label-3,5.391071e-01
4,glucose-6-phosphate,C6H10O5,"{'H': ['C', 'O17', 'O18']}",1_1,-9.814918e-04,0.00,H2-label-4,0.000000e+00
5,glucose-6-phosphate,C6H10O5,"{'H': ['C', 'O17', 'O18']}",1_1,1.759682e-06,0.00,H2-label-5,1.759682e-06
6,glucose-6-phosphate,C6H10O5,"{'H': ['C', 'O17', 'O18']}",1_1,-3.152301e-09,0.00,H2-label-6,0.000000e+00
7,glucose-6-phosphate,C6H10O5,"{'H': ['C', 'O17', 'O18']}",1_1,1.822857e-11,0.00,H2-label-7,1.822857e-11
8,glucose-6-phosphate,C6H10O5,"{'H': ['C', 'O17', 'O18']}",1_1,7.570558e-12,0.00,H2-label-8,7.570558e-12
9,glucose-6-phosphate,C6H10O5,"{'H': ['C', 'O17', 'O18']}",1_1,-1.043700e-11,0.00,H2-label-9,0.000000e+00


Calculating fractional enrichments, merging all data into a single file and saving as 'auto_detect_dual_label_isotope_ppm7.csv'

In [27]:
frac_enr_df = fractional_enrichment(na_corr_df)
frac_enr_df

,Sample,Name,Label,Formula,Pool_total,Fractional enrichment
0,A12_1,6-phospho-D-gluconate,C12 PARENT,C6H13O10P,27411.885482,0.011401
1,A12_1,6-phospho-D-gluconate,C13-label-1,C6H13O10P,27411.885482,0.021351
2,A12_1,6-phospho-D-gluconate,C13-label-2,C6H13O10P,27411.885482,0.965225
3,A12_1,6-phospho-D-gluconate,C13-label-3,C6H13O10P,27411.885482,0.000000
4,A12_1,6-phospho-D-gluconate,C13-label-4,C6H13O10P,27411.885482,0.002023
5,A12_1,6-phospho-D-gluconate,C13-label-5,C6H13O10P,27411.885482,0.000000
6,A12_1,6-phospho-D-gluconate,C13-label-6,C6H13O10P,27411.885482,0.000000
7,A12_1,fructose-1-6-bisphosphate,C12 PARENT,C6H14O12P2,445750.481190,0.021801
8,A12_1,fructose-1-6-bisphosphate,C13-label-1,C6H14O12P2,445750.481190,0.021857
9,A12_1,fructose-1-6-bisphosphate,C13-label-2,C6H14O12P2,445750.481190,0.898329


In [28]:
output_df = merge_multiple_dfs([merged_df, na_corr_df, frac_enr_df])
output_df

,Name,Label,Formula,Sample,Intensity_x,Unlabeled Fragment,Indistinguishable_isotope,NA Corrected,Intensity_y,NA Corrected with zero,Pool_total,Fractional enrichment
0,glucose-6-phosphate,C12 PARENT,C6H13O9P,A12_1,85751.28,glucose-6-phosphate,"{'C': ['H', u'O']}",93633.540138,85751.28,93633.540138,477270.063068,0.196186
1,glucose-6-phosphate,C13-label-1,C6H13O9P,A12_1,12179.52,glucose-6-phosphate,"{'C': ['H', u'O']}",6689.416355,12179.52,6689.416355,477270.063068,0.014016
2,glucose-6-phosphate,C13-label-2,C6H13O9P,A12_1,345720.09,glucose-6-phosphate,"{'C': ['H', u'O']}",367186.961554,345720.09,367186.961554,477270.063068,0.769348
3,glucose-6-phosphate,C13-label-3,C6H13O9P,A12_1,12830.81,glucose-6-phosphate,"{'C': ['H', u'O']}",-4188.956976,12830.81,0.000000,477270.063068,0.000000
4,glucose-6-phosphate,C13-label-4,C6H13O9P,A12_1,15879.57,glucose-6-phosphate,"{'C': ['H', u'O']}",9760.145020,15879.57,9760.145020,477270.063068,0.020450
5,glucose-6-phosphate,C13-label-5,C6H13O9P,A12_1,0.00,glucose-6-phosphate,"{'C': ['H', u'O']}",-498.054193,0.00,0.000000,477270.063068,0.000000
6,6-phospho-D-gluconate,C12 PARENT,C6H13O10P,A12_1,285.51,6-phospho-D-gluconate,"{'C': ['H', u'O']}",312.513495,285.51,312.513495,27411.885482,0.011401
7,6-phospho-D-gluconate,C13-label-1,C6H13O10P,A12_1,560.53,6-phospho-D-gluconate,"{'C': ['H', u'O']}",585.276584,560.53,585.276584,27411.885482,0.021351
8,6-phospho-D-gluconate,C13-label-2,C6H13O10P,A12_1,24736.69,6-phospho-D-gluconate,"{'C': ['H', u'O']}",26458.635037,24736.69,26458.635037,27411.885482,0.965225
9,6-phospho-D-gluconate,C13-label-3,C6H13O10P,A12_1,615.21,6-phospho-D-gluconate,"{'C': ['H', u'O']}",-632.323182,615.21,0.000000,27411.885482,0.000000


In [29]:
output_df.to_csv('C_sample_output.csv')